## 네이버 영화평

- 네이버 영화에서 분석하고 싶은 영화 페이지로 들어감
- 네티즌 별점 클릭
- 페이지 번호 우클릭 후 주소 복사

In [9]:
url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150689&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={}'

In [10]:
import requests

In [11]:
import lxml.html

In [12]:
res = requests.get(url.format(2))

In [13]:
root = lxml.html.fromstring(res.text)

In [18]:
for item in root.cssselect('div.score_result div.star_score em'):
    print(item.text)

8
1
6
1
10
1
3
4
1
6


### 리뷰 수집

In [19]:
import requests
import lxml.html

# 리뷰와 별점을 모을 빈 리스트를 만든다
reviews = []
scores = []

for page in range(1, 30):  # 1~29페이지까지 반복
    res = requests.get(url.format(page))  # 각 페이지에 접속한다
    root = lxml.html.fromstring(res.text)  # html을 처리한다

    # 리뷰를 가져와 reviews에 추가한다
    for review in root.cssselect('.score_reple p'):
        reviews.append(review.text_content())

    # 별점을 가져와 scores에 추가한다
    for score in root.cssselect('.score_result .star_score em'):
        scores.append(int(score.text_content()))

### 표 만들기

In [20]:
import pandas as pd

df = pd.DataFrame({
    'score': scores, 
    'review': reviews
})

In [21]:
df.head()

,score,review
0,6,영화는 괜찮음 생각보다. 근데 얘네는 왜 핵얘기 나올때마다 지들이 진주만 공격한건 ...
1,10,관람객이시하라 사토미 예쁘다
2,1,이래서 일본은 애니를 보나보다.
3,1,야 진짜 무슨 구연동화냐 겁나 허접한게 느껴짐
4,3,호불호 소리는 많이 들었지만 난 불호다. 고질라가 때려부수는거 보러갔더니 일본애들이...


### 긍/부정 표시

In [22]:
import numpy as np

In [23]:
df['sentiment'] = np.where(df['score'] > 5, 1, 0)

In [24]:
df.head()

,score,review,sentiment
0,6,영화는 괜찮음 생각보다. 근데 얘네는 왜 핵얘기 나올때마다 지들이 진주만 공격한건 ...,1
1,10,관람객이시하라 사토미 예쁘다,1
2,1,이래서 일본은 애니를 보나보다.,0
3,1,야 진짜 무슨 구연동화냐 겁나 허접한게 느껴짐,0
4,3,호불호 소리는 많이 들었지만 난 불호다. 고질라가 때려부수는거 보러갔더니 일본애들이...,0


### 저장

In [25]:
df.to_csv('movie_review.csv', encoding='utf8', index=False)

### 불러오기

In [26]:
df = pd.read_csv('movie_review.csv', encoding='utf8')

## 한글, 알파벳, 숫자 제외한 문자 제거

Word Piece Model 
- 자주 나오는 뭉치를 한 덩어리로 인식. ex. 노잼 이 많이 나오면 한 단어로 인식. 
- 형태소 분석과 달리 조사를 따로 빼지 못함. 
- 감성 분석에서 조사가 들어가는 건 크게 상관 없음. 
- word count 시에는 영향을 미칠수도.

In [29]:
import re

In [31]:
def remove_non_word(text):
    """한글, 알파벳, 숫자를 제외한 문자를 제거"""
    #정규 표현식 이용해서 찾아바꾸기. 
    #^ 이후 제외 + 이런 것들
    return re.sub(r'[^가-힣A-z0-9]+', ' ', text)

In [32]:
remove_non_word('Wow, 정말 1도 재미 없다!')

'Wow 정말 1도 재미 없다 '

### WPM 학습

In [1]:
!pip install subword_nmt

You are using pip version 10.0.1, however version 19.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
from subword_nmt.learn_bpe import learn_bpe
import io

with open('영화평BPE.txt', 'w', encoding='utf8') as outfile:
    infile = io.StringIO(remove_non_word(' '.join(df['review'])))
    learn_bpe(infile, outfile, 1000)

NameError: name 'remove_non_word' is not defined

In [21]:
from subword_nmt.apply_bpe import BPE

with open('영화평BPE.txt', encoding='utf8') as f:
    bpe = BPE(f, separator='~')

### TDM 만들기

In [22]:
def tokenizer_wpm(text):
    text = remove_non_word(text)
    tokens = bpe.process_line(text)
    tokens = tokens.split()
    return [t for t in tokens
            if (not t.endswith('~') and len(t) > 1) or len(t) > 2]

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

In [25]:
cv_wpm = CountVectorizer(max_features=1000, tokenizer=tokenizer_wpm)

In [26]:
tdm = cv_wpm.fit_transform(df['review'])

### 토큰 빈도

In [27]:
freq = pd.DataFrame({
    'word': cv_wpm.get_feature_names(),
    'n': tdm.sum(axis=0).flat
})

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\dtypes\dtypes.py:271: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if string == 'category':


In [28]:
freq.sort_values('n').tail(10)

,word,n
824,하고,20
15,cg~,20
578,일본~,20
57,관람객~,24
499,으로,26
446,영화~,33
834,하는,35
577,일본,43
445,영화,52
46,고질라,55


### 데이터 분할

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
x_train, x_test, y_train, y_test = train_test_split(
    tdm, df['sentiment'], test_size=.2, random_state=1234)

### 학습

In [34]:
from sklearn.linear_model import LogisticRegressionCV

In [35]:
model = LogisticRegressionCV(random_state=1234)

In [36]:
model.fit(x_train, y_train)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


LogisticRegressionCV(Cs=10, class_weight=None, cv='warn', dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=1234, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=0)

### 성능 평가

In [39]:
from sklearn.metrics import accuracy_score

In [40]:
y_pred = model.predict(x_test)

In [41]:
accuracy_score(y_test, y_pred)

0.6551724137931034

### 계수 분석

In [42]:
word_coef = pd.DataFrame({
    'word': cv_wpm.get_feature_names(),
    'coef': model.coef_.flat
})

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\dtypes\dtypes.py:271: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if string == 'category':


In [48]:
word_coef.sort_values('coef').head(10)

,word,coef
77,그냥,-0.545244
824,하고,-0.520906
248,보다,-0.485059
343,아깝다,-0.391294
163,들의,-0.353099
48,고질라가,-0.333812
256,볼거~,-0.320252
868,해서,-0.318450
133,다가,-0.298682
56,관람~,-0.297463


In [47]:
word_coef.sort_values('coef').tail(10)

,word,coef
541,이상~,0.354573
50,고질라를,0.364588
762,최고~,0.366515
809,풍자~,0.369840
499,으로,0.385134
59,관료제,0.395730
49,고질라는,0.435536
249,보다는,0.460171
254,보여~,0.540612
57,관람객~,1.026282
